In [10]:
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm

from scipy.sparse import lil_matrix

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.impute import SimpleImputer

In [2]:
from developed_methods import *

In [3]:
import scipy as sp
from scipy.stats import chi2

def mahalanobis_method(df):
    #M-Distance
    x_minus_mu = df - np.mean(df)
    cov = np.cov(df.values.T)                           #Covariance
    inv_covmat = sp.linalg.inv(cov)                     #Inverse covariance
    left_term = np.dot(x_minus_mu, inv_covmat) 
    mahal = np.dot(left_term, x_minus_mu.T)
    md = np.sqrt(mahal.diagonal())
    
    #Flag as outlier
    outlier = []
    #Cut-off point
    C = np.sqrt(chi2.ppf((1-0.001), df=df.shape[1]))    #degrees of freedom = number of variables
    for index, value in enumerate(md):
        if value > C:
            outlier.append(index)
        else:
            continue
    return outlier, md

# save the predicted ratings to csv file
def save_csv(df, folder_path, method):
    nowTime = datetime.now().strftime("%Y-%m-%d_%H-%M")
    fileName = "{folder_path}/{method}_{nowTime}.csv".format(folder_path = folder_path, method = method, nowTime = nowTime)
    df.to_csv(fileName, index = False)


In [4]:
# Load dataset
train_rating = pd.read_csv("../data/train_rating.csv")
test_pair = pd.read_csv("../data/test_pair.csv")

item_feat = pd.read_csv("../data/item_feats.csv")
user_feat = pd.read_csv("../data/user_feats.csv")

sub = pd.read_csv('../predict/sample_submission.csv')

In [5]:
# UserID
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(np.append(train_rating['UserId'], test_pair["UserId"]), user_feat["UserId"]))

user_feat['UserId'] = le_user.transform(user_feat["UserId"])
test_pair["UserId"] = le_user.transform(test_pair["UserId"])
train_rating['UserId'] = le_user.transform(train_rating["UserId"])

# ItemID
le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(np.append(train_rating['ItemId'], test_pair["ItemId"]), item_feat["ItemId"]))

item_feat['ItemId'] = le_item.transform(item_feat["ItemId"])
test_pair["ItemId"] = le_item.transform(test_pair["ItemId"])
train_rating['ItemId'] = le_item.transform(train_rating["ItemId"])

#Inf value
user_feat.loc[np.isinf(user_feat['V1']),'V1']=-3
item_feat.loc[np.isinf(item_feat['V2']),'V2']=2

# Missing data
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(user_feat['V1'].values.reshape(-1, 1))
user_feat['V1'] = imp_mean.transform(user_feat['V1'].values.reshape(-1, 1))

In [6]:
# tran_pair, train_rating
train_pairs = train_rating[['UserId', 'ItemId']].values
train_ratings = train_rating['rating'].values
train_pair=train_rating.drop(columns='rating')

# test_pair
test_pairs = test_pair[['UserId', 'ItemId']].values

# number of users and items
# n_user, n_item = len(le_item.classes_), len(le_item.classes_)

In [7]:
n_user, n_item = max(train_pairs[:,0].max(), test_pairs[:,0].max())+1, max(train_pairs[:,1].max(), test_pairs[:,1].max())+1

In [8]:
class min_max_adj:
    def __init__(self, train_rating):
        self.min = np.min(train_rating)
        self.max = np.max(train_rating)
        self.true_rating = train_rating
    
    def adjust(self, pred_rating):
        pred_rating_adjusted = pred_rating.copy()
        pred_rating_adjusted[pred_rating > self.max] = self.max
        pred_rating_adjusted[pred_rating < self.min] = self.min
        return pred_rating_adjusted

    def rmse(self, pred_rating):
        return np.sqrt(np.mean((pred_rating - self.true_rating)**2))

adjustment = min_max_adj(train_rating["rating"])
print("Minmium and maximum:", [adjustment.min, adjustment.max])

Minmium and maximum: [0.0, 5.0]


In [41]:
## generate cont feats for users
user_pd = pd.merge(left=train_rating.groupby('UserId')['rating'].mean(), 
				   right=train_rating.groupby('UserId')['rating'].count(), on='UserId')
user_pd.columns = ['rating_mean', 'rating_count']
user_pd = pd.merge(left = user_feat, right = user_pd, on = "UserId", how = "outer") # using outer join

## handle missing data
# if the user has no rating record, set rating_count = 0
user_pd.fillna(value = {"rating_count": 0}, inplace = True)
# if the rating_mean is missing, then use global mean
imp_mean.fit(user_pd)
user_pd = pd.DataFrame(imp_mean.transform(user_pd), columns = user_pd.columns)

## generate cont feats for items
item_rating_pd = pd.merge(left=train_rating.groupby('ItemId')['rating'].mean(), 
						  right=train_rating.groupby('ItemId')['rating'].count(), on='ItemId')
item_rating_pd.columns	= ['rating_mean', 'rating_count']
item_pd = pd.merge(left=item_feat, right=item_rating_pd, on='ItemId', how = "outer") # using outer join

## handle missing data
# if the item has no rating record, set rating_count = 0
item_pd.fillna(value = {"rating_count": 0}, inplace = True)
# if the rating_mean is missing, then use global mean
imp_mean.fit(item_pd)
item_pd = pd.DataFrame(imp_mean.transform(item_pd), columns = item_pd.columns)


print('#######################################################')
print('########## 10 random samples for users feats ##########')
print('#######################################################')

print(user_pd.sample(10))
print('#######################################################')
print('########## 10 random samples for items feats ##########')
print('#######################################################')

print(item_pd.sample(10))

#######################################################
########## 10 random samples for users feats ##########
#######################################################
        UserId        V1      V2      V3     V4  rating_mean  rating_count
20664  14405.0  2.766651  4277.0  1022.0  121.0     2.211891           0.0
7429    1893.0  3.423729  4611.0   883.0  154.0     2.211891           0.0
19990   9333.0  3.423729  1530.0   273.0  228.0     2.211891           0.0
13378  18201.0  3.807812  1410.0   262.0  330.0     2.211891           0.0
17075   3992.0  3.423729    64.0   396.0   86.0     2.211891           0.0
26454  23129.0  3.423729  5624.0   262.0  330.0     2.211891           0.0
15249  18821.0  3.320486  5608.0   588.0  121.0     2.211891           0.0
12120   1998.0  3.230220  1030.0   123.0  325.0     2.211891           0.0
12647  15880.0  3.423729  6755.0   844.0  330.0     2.211891           0.0
6482   14257.0  3.423729   542.0  1521.0   22.0     2.211891           0.0
#######

In [42]:
## pre-processing for users
user_cont = ["V1", "V2", "V3", "V4", "rating_mean", "rating_count"]
user_pd[user_cont] = preprocessing.StandardScaler().fit_transform(user_pd[user_cont])

## pre-processing for item
item_cont = ["V1", "V2", "V3", "rating_mean", "rating_count"]
item_pd[item_cont] = preprocessing.StandardScaler().fit_transform(item_pd[item_cont])


user_pd = user_pd.set_index('UserId', drop=True)
item_pd = item_pd.set_index('ItemId', drop=True)

print('#######################################################')
print('########## 10 random samples for users feats ##########')
print('#######################################################')
print(user_pd.sample(10))

print('#######################################################')
print('########## 10 random samples for items feats ##########')
print('#######################################################')
print(item_pd.sample(10))

#######################################################
########## 10 random samples for users feats ##########
#######################################################
               V1        V2        V3        V4   rating_mean  rating_count
UserId                                                                     
19188.0  0.567255  1.115762  0.477135 -1.068355 -5.564071e-16     -0.067335
9467.0  -0.174131 -0.895074 -1.019990  0.762978 -5.564071e-16     -0.067335
21423.0  0.000000 -0.747188  1.379421 -1.068355 -1.675014e+00      0.113023
2688.0   0.000000 -1.220848 -1.672755 -1.637908 -5.564071e-16     -0.067335
13564.0  0.467600 -0.895074 -1.019990  0.762978 -5.564071e-16     -0.067335
14437.0 -0.051990  1.278014  0.289995 -1.734294 -5.564071e-16     -0.067335
26377.0 -0.549315  0.015907 -0.079831  0.105801 -5.564071e-16     -0.067335
22673.0  0.000000  0.937030 -1.216042  0.762978  2.240353e+00      0.022844
1254.0   0.000000 -0.947469 -1.040041  0.719167 -5.564071e-16     -0.067

In [43]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input, Dropout, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from IPython.display import SVG
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [44]:
class SideNCF(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(SideNCF, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.itme_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )

        self.concatenate = layers.Concatenate()
        self.dense1 = layers.Dense(100, name='fc-1', activation='relu')
        self.dense2 = layers.Dense(50, name='fc-2', activation='relu')
        self.dense3 = layers.Dense(1, name='fc-3', activation='relu')

    def call(self, inputs):
        cont_feats = inputs[0]
        cate_feats = inputs[1]

        user_vector = self.user_embedding(cate_feats[:,0])
        itme_vector = self.itme_embedding(cate_feats[:,1])

        concatted_vec = self.concatenate([cont_feats, user_vector, itme_vector])
        fc_1 = self.dense1(concatted_vec)
        fc_2 = self.dense2(fc_1)
        fc_3 = self.dense3(fc_2)
        return fc_3

In [45]:
model = SideNCF(num_users=n_user, num_items=n_item, embedding_size=50)

metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

In [46]:
## find the continuous features and categorical features for user and item, respectively
item_cont, item_cate = ["V1", "V2", "V3", 'rating_mean', 'rating_count'], ['ItemId']
user_cont, user_cate = ["V1", "V2", "V3", "V4", 'rating_mean', 'rating_count'], ['UserId']

train_cont_feats = np.hstack((user_pd.loc[train_pairs[:,0]][user_cont], item_pd.loc[train_pairs[:,1]][item_cont]))
train_cate_feats = train_pairs.copy()

test_cont_feats = np.hstack((user_pd.loc[test_pairs[:,0]][user_cont], item_pd.loc[test_pairs[:,1]][item_cont]))
test_cate_feats = test_pairs.copy()

In [47]:
callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=[train_cont_feats, train_cate_feats],
    y=train_ratings,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=.2,
)

Epoch 1/50
504/504 [==============================] - 16s 29ms/step - loss: 2.6807 - mae: 0.9918 - rmse: 1.6188 - val_loss: 1.4861 - val_mae: 0.7722 - val_rmse: 1.2042
Epoch 2/50
504/504 [==============================] - 14s 28ms/step - loss: 1.4736 - mae: 0.7457 - rmse: 1.1997 - val_loss: 1.4725 - val_mae: 0.7373 - val_rmse: 1.2001
Epoch 3/50
504/504 [==============================] - 14s 28ms/step - loss: 1.4612 - mae: 0.7337 - rmse: 1.1947 - val_loss: 1.5574 - val_mae: 0.7438 - val_rmse: 1.2361
Epoch 4/50
504/504 [==============================] - 14s 27ms/step - loss: 1.4104 - mae: 0.7178 - rmse: 1.1731 - val_loss: 1.4271 - val_mae: 0.7299 - val_rmse: 1.1799
Epoch 5/50
504/504 [==============================] - 15s 30ms/step - loss: 1.4113 - mae: 0.7120 - rmse: 1.1705 - val_loss: 1.4330 - val_mae: 0.7188 - val_rmse: 1.1784
Epoch 6/50
504/504 [==============================] - 14s 28ms/step - loss: 1.4188 - mae: 0.7130 - rmse: 1.1716 - val_loss: 1.5547 - val_mae: 0.7535 - val_rmse:

In [49]:
pred_rating = model.predict([train_cont_feats, train_cate_feats]).flatten()
pred_rating = adjustment.adjust(pred_rating)
print(pred_rating)
print('rmse: SideNCF: %.3f' %rmse(train_ratings, pred_rating))

[0.        0.        0.        ... 0.        0.        4.5314155]
rmse: SideNCF: 0.854


In [51]:
# save the predicted ratings to csv file
def save_csv(df, folder_path, method):
    nowTime = datetime.now().strftime("%Y-%m-%d_%H-%M")
    fileName = "{folder_path}/{method}_{nowTime}.csv".format(folder_path = folder_path, method = method, nowTime = nowTime)
    df.to_csv(fileName, index = False)

sub = pd.read_csv('../predict/sample_submission.csv')

In [55]:
pred = model.predict([test_cont_feats, test_cate_feats]).flatten()
pred = adjustment.adjust(pred)
sub["rating"] = pred
save_csv(sub, "../predict", "NCF")